In [1]:
def read_file(day, ex=1):
    with open(f"inputs/day{day}-{ex}.txt", "r") as f:
        return f.readlines()

In [2]:
day=1
ex=1

last_number = None
increases = 0
for number in read_file(day, ex):
    if last_number and int(number) > last_number:
        increases += 1
    last_number = int(number)
print(increases)

1466


In [5]:
day=1
ex=2

window = [0, 0, 0]
increases = 0
for i, number in enumerate(read_file(day, 1)): # same input as 1
    
    if i >= 3 and int(number) > window[i % 3]:
        increases += 1
    window[i % 3] = int(number)
print(increases)

5
